# Introduction

## Objective:
The objective of this analysis is to investigate the relationships between various employee-related factors and attrition. We aim to perform a t-test to check for significant differences in monthly income between employees who left the company (`attrition = 'Yes'`) and those who stayed (`attrition = 'No'`). Additionally, regression analysis is performed to understand the relationship between job-related factors and monthly income.

## Methodology:
1. **Data Collection**: The data is fetched from a PostgreSQL database using `SQLAlchemy` and `psycopg2` to connect to the database.
2. **Exploratory Data Analysis (EDA)**: Basic statistical analysis and correlation assessments were performed.
3. **Hypothesis Testing**: A t-test is used to evaluate whether there is a significant difference in monthly income between employees who left and stayed.
4. **Regression Analysis**: We used an OLS regression model to evaluate the impact of job level and years at the company on monthly income.

---

In [1]:
pip install psycopg2 sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [2]:
from sqlalchemy import create_engine
import psycopg2
import pandas as pd

# Parametres
username = "postgres"         
password = "password"         
host = "localhost"            
port = "5432"                 
database = "HR_Analystics"    

# СString connection for SQLAlchemy
connection_string = f"postgresql://{username}:{password}@{host}:{port}/{database}"

# Check connection for SQLAlchemy
print("Testing connection with SQLAlchemy...")
try:
    engine = create_engine(connection_string)
    # Try simple querry.
    test_query = "SELECT 1;"
    with engine.connect() as conn:
        result = conn.execute(test_query)
        print("SQLAlchemy connection successful:", result.fetchall())
except Exception as e:
    print("SQLAlchemy connection failed:", e)

# Check connections with psycopg2
print("\nTesting connection with psycopg2...")
try:
    conn = psycopg2.connect(
        dbname=database,
        user=username,
        password=password,
        host=host,
        port=port
    )
    print("psycopg2 connection successful!")
    conn.close()
except Exception as e:
    print("psycopg2 connection failed:", e)

# Querry if connections successful
query = """
SELECT 
    e.EmpID,
    e.Age,
    e.AgeGroup,
    e.Attrition,
    e.DailyRate,
    e.DistanceFromHome,
    e.Education,
    ef.EducationFieldName AS EducationField,
    e.EmployeeNumber,
    e.EnvironmentSatisfaction,
    e.Gender,
    e.HourlyRate,
    e.JobInvolvement,
    e.JobLevel,
    jr.JobRoleName AS JobRole,
    e.JobSatisfaction,
    ms.MaritalStatusName AS MaritalStatus,
    e.MonthlyIncome,
    e.SalarySlab,
    e.MonthlyRate,
    e.NumCompaniesWorked,
    e.OverTime,
    e.PercentSalaryHike,
    e.PerformanceRating,
    e.RelationshipSatisfaction,
    e.StockOptionLevel,
    e.TotalWorkingYears,
    e.TrainingTimesLastYear,
    e.WorkLifeBalance,
    e.YearsAtCompany,
    e.YearsInCurrentRole,
    e.YearsSinceLastPromotion,
    e.YearsWithCurrManager,
    d.DepartmentName AS Department,
    bt.BusinessTravelName AS BusinessTravel
FROM Employees e
LEFT JOIN Departments d ON e.DepartmentID = d.DepartmentID
LEFT JOIN EducationFields ef ON e.EducationFieldID = ef.EducationFieldID
LEFT JOIN JobRoles jr ON e.JobRoleID = jr.JobRoleID
LEFT JOIN MaritalStatuses ms ON e.MaritalStatusID = ms.MaritalStatusID
LEFT JOIN BusinessTravels bt ON e.BusinessTravelID = bt.BusinessTravelID;
"""

print("\nExecuting query through SQLAlchemy...")
try:
    # Reada data with DataFrame
    df = pd.read_sql_query(query, engine)
    print("Query executed successfully! Here's the data:")
    print(df.head())
except Exception as e:
    print("Query execution failed:", e)

Testing connection with SQLAlchemy...
SQLAlchemy connection failed: Not an executable object: 'SELECT 1;'

Testing connection with psycopg2...
psycopg2 connection successful!

Executing query through SQLAlchemy...
Query executed successfully! Here's the data:
   empid  age agegroup  attrition  dailyrate  distancefromhome  education  \
0  RM297   18    18-25       True        230                 3          3   
1  RM302   18    18-25      False        812                10          3   
2  RM458   18    18-25       True       1306                 5          3   
3  RM728   18    18-25      False        287                 5          2   
4  RM829   18    18-25       True        247                 8          1   

  educationfield  employeenumber  environmentsatisfaction  ...  \
0  Life Sciences             405                        3  ...   
1        Medical             411                        4  ...   
2      Marketing             614                        2  ...   
3  Life Scien

In [3]:
df.head()

,empid,age,agegroup,attrition,dailyrate,distancefromhome,education,educationfield,employeenumber,environmentsatisfaction,...,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearsincurrentrole,yearssincelastpromotion,yearswithcurrmanager,department,businesstravel
0,RM297,18,18-25,True,230,3,3,Life Sciences,405,3,...,0,0,2,3,0,0,0,0.0,Research & Development,Travel_Rarely
1,RM302,18,18-25,False,812,10,3,Medical,411,4,...,0,0,2,3,0,0,0,0.0,Sales,Travel_Rarely
2,RM458,18,18-25,True,1306,5,3,Marketing,614,2,...,0,0,3,3,0,0,0,0.0,Sales,Travel_Frequently
3,RM728,18,18-25,False,287,5,2,Life Sciences,1012,2,...,0,0,2,3,0,0,0,0.0,Research & Development,Non-Travel
4,RM829,18,18-25,True,247,8,1,Medical,1156,3,...,0,0,0,3,0,0,0,0.0,Research & Development,Non-Travel


In [4]:
df.describe()

,age,dailyrate,distancefromhome,education,employeenumber,environmentsatisfaction,hourlyrate,jobinvolvement,joblevel,jobsatisfaction,...,performancerating,relationshipsatisfaction,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearsincurrentrole,yearssincelastpromotion,yearswithcurrmanager
count,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,...,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000
mean,36.923810,802.485714,9.192517,2.912925,1024.865306,2.721769,65.891156,2.729932,2.063946,2.728571,...,3.153741,2.712245,0.793878,11.279592,2.799320,2.761224,7.008163,4.229252,2.187755,4.076871
std,9.135373,403.509100,8.106864,1.024165,602.024335,1.093082,20.329428,0.711561,1.106940,1.102846,...,0.360824,1.081209,0.852077,7.780782,1.289271,0.706476,6.126525,3.623137,3.222430,3.498547
min,18.000000,102.000000,1.000000,1.000000,1.000000,1.000000,30.000000,1.000000,1.000000,1.000000,...,3.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,465.000000,2.000000,2.000000,491.250000,2.000000,48.000000,2.000000,1.000000,2.000000,...,3.000000,2.000000,0.000000,6.000000,2.000000,2.000000,3.000000,2.000000,0.000000,2.000000
50%,36.000000,802.000000,7.000000,3.000000,1020.500000,3.000000,66.000000,3.000000,2.000000,3.000000,...,3.000000,3.000000,1.000000,10.000000,3.000000,3.000000,5.000000,3.000000,1.000000,3.000000
75%,43.000000,1157.000000,14.000000,4.000000,1555.750000,4.000000,83.750000,3.000000,3.000000,4.000000,...,3.000000,4.000000,1.000000,15.000000,3.000000,3.000000,9.000000,7.000000,3.000000,7.000000
max,60.000000,1499.000000,29.000000,5.000000,2068.000000,4.000000,100.000000,4.000000,5.000000,4.000000,...,4.000000,4.000000,3.000000,40.000000,6.000000,4.000000,40.000000,18.000000,15.000000,17.000000


### 1. Correlation Analysis

In [5]:
# Select only numeric columns for correlation analysis
numeric_df = df.select_dtypes(include=['float64', 'int64'])

# Perform correlation analysis
correlation_matrix = numeric_df.corr()
correlation_matrix

,age,dailyrate,distancefromhome,education,employeenumber,environmentsatisfaction,hourlyrate,jobinvolvement,joblevel,jobsatisfaction,...,performancerating,relationshipsatisfaction,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearsincurrentrole,yearssincelastpromotion,yearswithcurrmanager
age,1.000000,0.010661,-0.001686,0.208034,-0.010145,0.010146,0.024287,0.029820,0.509604,-0.004892,...,0.001904,0.053535,0.037510,0.680381,-0.019621,-0.021490,0.311309,0.212901,0.216513,0.207127
dailyrate,0.010661,1.000000,-0.004985,-0.016806,-0.050990,0.018355,0.023381,0.046135,0.002966,0.030571,...,0.000473,0.007846,0.042143,0.014515,0.002453,-0.037848,-0.034055,0.009932,-0.033229,-0.022798
distancefromhome,-0.001686,-0.004985,1.000000,0.021042,0.032916,-0.016075,0.031131,0.008783,0.005303,-0.003669,...,0.027110,0.006557,0.044872,0.004628,-0.036942,-0.026556,0.009508,0.018845,0.010029,0.014863
education,0.208034,-0.016806,0.021042,1.000000,0.042070,-0.027128,0.016775,0.042438,0.101589,-0.011296,...,-0.024539,-0.009118,0.018422,0.148280,-0.025100,0.009819,0.069114,0.060236,0.054254,0.065704
employeenumber,-0.010145,-0.050990,0.032916,0.042070,1.000000,0.017621,0.035179,-0.006888,-0.018519,-0.046247,...,-0.020359,-0.069861,0.062227,-0.014365,0.023603,0.010309,-0.011240,-0.008416,-0.009019,0.007760
environmentsatisfaction,0.010146,0.018355,-0.016075,-0.027128,0.017621,1.000000,-0.049857,-0.008278,0.001212,-0.006784,...,-0.029548,0.007665,0.003432,-0.002693,-0.019359,0.027627,0.001458,0.018007,0.016194,0.004529
hourlyrate,0.024287,0.023381,0.031131,0.016775,0.035179,-0.049857,1.000000,0.042861,-0.027853,-0.071335,...,-0.002172,0.001330,0.050263,-0.002334,-0.008548,-0.004607,-0.019582,-0.024106,-0.026716,-0.002151
jobinvolvement,0.029820,0.046135,0.008783,0.042438,-0.006888,-0.008278,0.042861,1.000000,-0.012630,-0.021476,...,-0.029071,0.034297,0.021523,-0.005533,-0.015338,-0.014617,-0.021355,0.008717,-0.024184,0.037604
joblevel,0.509604,0.002966,0.005303,0.101589,-0.018519,0.001212,-0.027853,-0.012630,1.000000,-0.001944,...,-0.021222,0.021642,0.013984,0.782208,-0.018191,0.037818,0.534739,0.389447,0.353885,0.368217
jobsatisfaction,-0.004892,0.030571,-0.003669,-0.011296,-0.046247,-0.006784,-0.071335,-0.021476,-0.001944,1.000000,...,0.002297,-0.012454,0.010690,-0.020185,-0.005779,-0.019459,-0.003803,-0.002305,-0.018214,-0.031286


### 2. Hypothesis Testing

In [12]:
from scipy.stats import ttest_ind

# Make sure 'attrition' is treated as a categorical column, convert if necessary
df['attrition'] = df['attrition'].map({True: 'Yes', False: 'No'})

# Perform t-test for monthly income by attrition status
stayed = df[df['attrition'] == 'No']['monthlyincome']
left = df[df['attrition'] == 'Yes']['monthlyincome']

# Perform the t-test
t_stat, p_val = ttest_ind(stayed, left, nan_policy='omit')  # Ignore NaN values if any

# Display the test statistic and p-value
print("T-test Statistic:", t_stat)
print("P-value:", p_val)

T-test Statistic: 6.203935765608938
P-value: 7.147363985353811e-10


### 3. Regression Analysis

In [10]:
import statsmodels.api as sm

# Define independent and dependent variables for regression
X = df[['yearsatcompany', 'joblevel']]
y = df['monthlyincome']
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          monthlyincome   R-squared:                       0.903
Model:                            OLS   Adj. R-squared:                  0.903
Method:                 Least Squares   F-statistic:                     6838.
Date:                Thu, 21 Nov 2024   Prob (F-statistic):               0.00
Time:                        20:02:38   Log-Likelihood:                -12801.
No. Observations:                1470   AIC:                         2.561e+04
Df Residuals:                    1467   BIC:                         2.562e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const          -1844.9428     81.213    -22.717      0.000   -2004.250   -1685.636
yearsatcompany     6.5892      7.390      0.892      0.373      -7.907      21.085
joblevel        4022.2457     40.902     98.340      0.000    3942.014    4102.477
==============================================================================
Omnibus:                        1.987   Durbin-Watson:                   1.973
Prob(Omnibus):                  0.370   Jarque-Bera (JB):                2.004
Skew:                          -0.018   Prob(JB):                        0.367
Kurtosis:                       3.177   Cond. No.                         21.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Overall Output for Completed Work

## 1. **SQL Database Connection**  
- **psycopg2 Connection:**
    - **Success**: The connection using `psycopg2` was successful.
    - **Key Finding**: psycopg2 successfully connected to the PostgreSQL database with the provided credentials.

## 2. **Hypothesis Testing**

- **T-Test for Monthly Income by Attrition Status:**
    - **T-test Statistic**: 6.203935765608938
    - **P-value**: 7.147363985353811e-10
    - **Interpretation**: The very low p-value (<0.05) indicates that there is a statistically significant difference between the monthly income of employees who stayed and those who left the company.
  
    - **Key Finding**: Employees who left the company had significantly different monthly income when compared to employees who stayed, with a very strong statistical relationship.

## 3. **Regression Analysis**

- **OLS Regression Results for Predicting Monthly Income:**
    - **R-squared**: 0.903 (indicating that 90.3% of the variation in monthly income can be explained by the model).
    - **Adjusted R-squared**: 0.903 (suggests a good fit, considering the number of predictors).
    - **F-statistic**: 6838, with a p-value of 0.00 (indicating that the model is significant overall).
  
    - **Coefficients:**
        - **Job Level**: Significant predictor of monthly income with a coefficient of 4022.2457 (p-value < 0.05).
        - **Years at Company**: Not a significant predictor with a p-value of 0.373, indicating weaker evidence for its relationship with monthly income.

    - **Key Finding**: **Job level** is a strong predictor of monthly income, while **years at company** has a weaker association, and does not contribute significantly to explaining income variation.

## 4. **Summary of Key Findings:**

- **SQL Database Connection**: 
  - psycopg2 was successful, but SQLAlchemy had an issue due to the query used.

- **Hypothesis Testing**:
  - The monthly income of employees who stayed vs. left is significantly different, with a p-value of **7.15e-10**.
  
- **Regression Analysis**:
  - The model explains 90.3% of the variance in **monthly income**, with **job level** being a significant predictor. **Years at company** did not significantly contribute to income variation.

---
